# Proof of concept of Imviz requirements using glupyter/bqplot

We start off by silencing warnings that can happen when loading data as well as deprecation warnings, for clarity:

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
from urllib.request import urlretrieve

urlretrieve('http://www.astropy.org/astropy-data/galactic_center/gc_2mass_j.fits', filename='2mass_j.fits')
urlretrieve('http://www.astropy.org/astropy-data/galactic_center/gc_msx_e.fits', filename='msx_e.fits')

('msx_e.fits', <http.client.HTTPMessage at 0x7f8d99980520>)

We start off by looking at some of the basic features:

In [3]:
from jdaviz import Imviz
import matplotlib.pyplot as plt

imviz = Imviz()
imviz.load_data('2mass_j.fits')
imviz.load_data('msx_e.fits')

viewer = imviz.app.get_viewer('viewer-1')

imviz.app

Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [4]:
viewer.state.layers[0].stretch = 'sqrt'

IndexError: list index out of range

the colormap:

In [ ]:
viewer.state.layers[0].cmap = plt.cm.viridis

the limits via the percentile option:

In [ ]:
viewer.state.layers[0].percentile = 90

or the limits directly:

In [ ]:
viewer.state.layers[0].v_min = -10
viewer.state.layers[0].v_max = +100

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.app.get_subsets_from_viewer('viewer-1')

In [ ]:
regions

Since the region is an astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='subpixels')

In [ ]:
plt.imshow(mask.to_image((2048, 2048)), origin='lower')